In [1]:
import pandas as pd
import numpy as np

import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)


In [3]:
fake_df = pd.read_csv("Fake.csv")
true_df = pd.read_csv("True.csv")


In [12]:
print(
    fake_df.head(),
    fake_df.info(),
    true_df.head(),
    true_df.info()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB
                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even

In [13]:
fake_df["label"] = 0   # Fake news
true_df["label"] = 1   # Real news


In [14]:
df = pd.concat([fake_df, true_df], axis=0)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [15]:
df["text"] = df["title"].fillna("") + " " + df["text"].fillna("")


In [17]:
print(df["label"].value_counts(), df["text"].str.len().describe())

label
0    23481
1    21417
Name: count, dtype: int64 count    44898.000000
mean      2550.221413
std       2172.578772
min         31.000000
25%       1318.000000
50%       2270.000000
75%       3189.000000
max      51893.000000
Name: text, dtype: float64


In [18]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB
None


In [19]:
df["combined_text"] = (
    df["title"] + " " +
    df["text"] + " " +
    df["subject"]
)


In [20]:
X = df["combined_text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [21]:
import re
import string

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"<.*?>", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text


In [22]:
X_train = X_train.apply(clean_text)
X_test = X_test.apply(clean_text)


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    max_df=0.9,
    min_df=10,
    stop_words=None
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [24]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_vec, y_train)

y_pred_lr = lr.predict(X_test_vec)


In [25]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))
confusion_matrix(y_test, y_pred_lr)


Logistic Regression Accuracy: 0.989086859688196
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4696
           1       0.99      0.99      0.99      4284

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



array([[4639,   57],
       [  41, 4243]])

In [26]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train_vec, y_train)

y_pred_nb = nb.predict(X_test_vec)

print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))


Naive Bayes Accuracy: 0.9606904231625836
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      4696
           1       0.96      0.96      0.96      4284

    accuracy                           0.96      8980
   macro avg       0.96      0.96      0.96      8980
weighted avg       0.96      0.96      0.96      8980



In [27]:
from sklearn.svm import LinearSVC

svm = LinearSVC()
svm.fit(X_train_vec, y_train)

y_pred_svm = svm.predict(X_test_vec)

print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


SVM Accuracy: 0.9973273942093541
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4696
           1       1.00      1.00      1.00      4284

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [30]:
feature_names = vectorizer.get_feature_names_out()
coeffs = lr.coef_[0]

top_fake = sorted(zip(coeffs, feature_names))[:15]
top_real = sorted(zip(coeffs, feature_names), reverse=True)[:15]

top_fake, top_real


([(np.float64(-12.04110665488018), 'video'),
  (np.float64(-6.8881020854202015), 'is'),
  (np.float64(-6.5626385627885755), 'politics'),
  (np.float64(-6.367340187269409), 'this'),
  (np.float64(-5.977889002586983), 'news'),
  (np.float64(-5.868373379899745), 'hillary'),
  (np.float64(-5.776365657010708), 'you'),
  (np.float64(-5.743969499290638), 'that'),
  (np.float64(-5.275863721743722), 'just'),
  (np.float64(-5.180132539203029), 'via'),
  (np.float64(-5.093470465383218), 'obama'),
  (np.float64(-4.882559512168196), 'leftnews'),
  (np.float64(-4.20389496452555), 'gop'),
  (np.float64(-3.9982304018487835), 'president trump'),
  (np.float64(-3.9658085235771634), 'watch')],
 [(np.float64(19.353980654306675), 'said'),
  (np.float64(14.203668911408835), 'reuters'),
  (np.float64(9.371317687762094), 'politicsnews'),
  (np.float64(7.283041198227659), 'worldnews'),
  (np.float64(6.347477446554246), 'said on'),
  (np.float64(5.959541986704245), 'washington reuters'),
  (np.float64(5.8622069

In [31]:
sample_texts = [
    "BREAKING: You won’t believe what the government is hiding from you!",
    "WASHINGTON (Reuters) - The president said on Tuesday that new policies will be introduced."
]

sample_clean = [clean_text(t) for t in sample_texts]
sample_vec = vectorizer.transform(sample_clean)

svm.predict(sample_vec)


array([0, 1])

In [32]:
y_shuffled = y_train.sample(frac=1, random_state=42)

svm_fake = LinearSVC()
svm_fake.fit(X_train_vec, y_shuffled)

y_fake_pred = svm_fake.predict(X_test_vec)
accuracy_score(y_test, y_fake_pred)


0.5035634743875278

In [33]:
X_text_only = df["text"]

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_text_only, y, test_size=0.2, random_state=42, stratify=y
)

X_train_t = X_train_t.apply(clean_text)
X_test_t = X_test_t.apply(clean_text)

vec_t = TfidfVectorizer(ngram_range=(1,2), min_df=10)
X_train_t_vec = vec_t.fit_transform(X_train_t)
X_test_t_vec = vec_t.transform(X_test_t)

svm_t = LinearSVC()
svm_t.fit(X_train_t_vec, y_train_t)

accuracy_score(y_test_t, svm_t.predict(X_test_t_vec))


0.9962138084632517